# Add human-in-the-loop controls
- Doc: https://langchain-ai.github.io/langgraph/tutorials/get-started/4-human-in-the-loop/

In [ ]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("google_genai:gemini-2.5-flash")

## 1. Add the human_assistance tool

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.types import Command, interrupt


# 에이전트의 상태 정의 클래스
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 상태 기반 워크플로우 생성
graph_builder = StateGraph(State)

# 챗봇이 인간의 개입을 요청할 때 호출됨 (도구)
@tool
def human_assistance(query: str) -> str:
    """Request assistance from a human."""
    # interrupt: 워크플로우를 일시 중지하고 인간의 응답을 기다림
    human_response = interrupt({"query": query})
    # 인간의 응답에서 data 반환
    return human_response["data"]

# 웹 검색을 도구
tool = TavilySearch(max_results=2)
# 도구 리스트
tools = [tool, human_assistance]

# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools(tools)

# chatbot 노드 함수
def chatbot(state: State):
    message = llm_with_tools.invoke(state["messages"])
    # 병렬 도구 호출 비활성화: 인터럽트 후 툴 중복 호출 방지
    assert len(message.tool_calls) <= 1
    return {"messages": [message]}

# 워크플로우에 chatbot 노드 추가
graph_builder.add_node("chatbot", chatbot)

# tool 노드 워크플로우에 추가
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

# 조건부 라우팅: tools로 이동하거나 END로 이동 (종료)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# tools 노드 실행 후 chatbot 노드로 다시 이동 (도구 결과 처리)
graph_builder.add_edge("tools", "chatbot")

# 워크플로우 시작점에서 chatbot 노드로 이동
graph_builder.add_edge(START, "chatbot")

## 2. Compile the graph

In [ ]:
memory = MemorySaver()
# 워크플로우 컴파일, 실행 가능한 워크플로우 생성.
# memory checkpointer 사용
graph = graph_builder.compile(checkpointer=memory)

## 3. Visualize the graph (optional)

In [ ]:
from IPython.display import Image, display

try:
    # 워크플로우 그래프를 이미지로 시각화하여 출력
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

## 4. Prompt the chatbot

In [ ]:
user_input = (
    "AI agent를 만드는 데 전문가의 도움이 필요해. "
    "도움을 요청해 줘"
)
# 대화 세션을 구분하기 위한 설정
config = {"configurable": {"thread_id": "1"}}

# 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config=config,
    stream_mode="values",  # values: 각 노드가 실행을 마친 후 결과 스트리밍
)
# 각 event의 마지막 메시지 출력
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
# 지정된 config의 워크플로우 상태 확인
snapshot = graph.get_state(config)
# snapshot.next: 다음에 실행될 노드
# 출력: ['tools'] (human_assistance 호출로 인해 tools 노드에서 일시 정지 상태)
snapshot.next

## 5. Resume execution

In [ ]:
# 인간의 응답을 가상으로 생성
human_response = (
    "저희 전문가들이 도와드리겠습니다!"
    " 에이전트를 구축하려면 LangGraph를 확인해 보세요."
    " 단순한 자율 에이전트보다 훨씬 안정적이고 확장성이 뛰어납니다."
)

# Command 객체의 resume에 인간 응답 저장
human_command = Command(resume={"data": human_response})

# 워크플로우를 스트리밍 모드로 실행 (인간 응답 처리)
events = graph.stream(
    human_command,
    config,
    stream_mode="values",  # values: 각 노드가 실행을 마친 후 결과를 스트리밍
)
# 각 event의 마지막 메시지 출력
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()